In [1]:
import os
import cv2
import numpy as np
import pandas as pd

from PIL import Image
from typing import Union

def breaker(num: int = 50, char: str = "*") -> None:
    print("\n" + num*char + "\n")

In [2]:
def get_image(path: str, size: int=224) -> np.ndarray:
    image = cv2.imread(path, cv2.IMREAD_COLOR)
    if image is None: 
        image = np.asarray(Image.open(path))
        if len(image.shape) == 2:
            image = np.concatenate((
                np.expand_dims(image, axis=2),
                np.expand_dims(image, axis=2),
                np.expand_dims(image, axis=2),
            ), axis=2)
    image = cv2.cvtColor(src=image, code=cv2.COLOR_BGR2RGB)
    return cv2.resize(src=image, dsize=(size, size), interpolation=cv2.INTER_AREA)


def get_statistics(filepaths: np.ndarray) -> dict:

    sizes: list = [224, 384, 512]
    stats: dict = dict()
    means: dict = dict()
    stds: dict  = dict()
        
    for size in sizes:
        total_num_of_images: int = 0
        r_mean, g_mean, b_mean, r_std, g_std, b_std = 0.0, 0.0, 0.0, 0.0, 0.0, 0.0
        for filepath in filepaths:
            image = get_image(filepath, size)
            if image is not None:
                r_mean += image[:, :, 0].mean()
                g_mean += image[:, :, 1].mean()
                b_mean += image[:, :, 2].mean()
                r_std += image[:, :, 0].std()
                g_std += image[:, :, 1].std()
                b_std += image[:, :, 2].std()
                total_num_of_images += 1
            else:
                continue
        
        means[f"{size}"] = [
            r_mean / total_num_of_images / 255, 
            g_mean / total_num_of_images / 255, 
            b_mean / total_num_of_images / 255,
        ]
        
        stds[f"{size}"]  = [
            r_std / total_num_of_images / 255, 
            g_std / total_num_of_images / 255, 
            b_std / total_num_of_images / 255,
            ]
    
    stats["means"] = means
    stats["stds"] = stds
    
    return stats


def print_statistics(stats: dict) -> None:
    breaker()
    print("Means".upper())
    breaker()
    for k, v in stats["means"].items():
        print(f"{k} x {k} Means\n")
        for i in range(len(v)):
            if i == 0:
                print(f"Red   : {v[i]:.5f}")
            if i == 1:
                print(f"Green : {v[i]:.5f}")
            if i == 2:
                print(f"Blue  : {v[i]:.5f}")
        breaker()
    
    breaker()
    print("Stds".upper())
    breaker()
    for k, v in stats["stds"].items():
        print(f"{k} x {k} Stds\n")
        for i in range(len(v)):
            if i == 0:
                print(f"Red   : {v[i]:.5f}")
            if i == 1:
                print(f"Green : {v[i]:.5f}")
            if i == 2:
                print(f"Blue  : {v[i]:.5f}")
        breaker()

In [3]:
df = pd.read_csv("../input/fdis-dataframe/dataframe.csv")

stats = get_statistics(df.filepaths.copy().values)
print_statistics(stats)

Premature end of JPEG file
Premature end of JPEG file
Premature end of JPEG file



**************************************************

MEANS

**************************************************

224 x 224 Means

Red   : 0.65967
Green : 0.62907
Blue  : 0.59212

**************************************************

384 x 384 Means

Red   : 0.65966
Green : 0.62906
Blue  : 0.59210

**************************************************

512 x 512 Means

Red   : 0.65965
Green : 0.62905
Blue  : 0.59209

**************************************************


**************************************************

STDS

**************************************************

224 x 224 Stds

Red   : 0.23083
Green : 0.23947
Blue  : 0.26350

**************************************************

384 x 384 Stds

Red   : 0.23390
Green : 0.24220
Blue  : 0.26589

**************************************************

512 x 512 Stds

Red   : 0.23497
Green : 0.24317
Blue  : 0.26674

**************************************************

